In [3]:
from keras.models import Model
from keras.layers import Input , Dense , GRU , LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
import numpy as np
import os



!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

--2020-06-21 12:44:38--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz [following]
--2020-06-21 12:44:38--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz.1’

download.php?f=Open 100%[===================>]   3.41G  23.3MB/s    in 2m 35s  

2020-06-21 12:47:15 (22.5 MB/s) - ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.

In [4]:
!gunzip -k '/content/download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz'
!mkdir lines

!split -a 3 -l 100000 '/content/download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en' lines/lines-





gzip: /content/download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en already exists; do you wish to overwrite (y or n)? ^C
mkdir: cannot create directory ‘lines’: File exists
^C


In [5]:

f = open('/content/drive/My Drive/lines-abu.txt' ,'r')
for i,line in enumerate(f):
  if i%2==0:
    x.append(line)
  else:
    y.append(line)



In [6]:
print(len(x))

from keras.models import Model
from keras.layers import Input , Dense , GRU , LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
import numpy as np
import os

num_samples = 40000


max_encoder_seq_length = 15
max_decoder_seq_length = 15


print(len(y))

print(x[:10])
print(y[:10])

tokenizer = Tokenizer(num_words=num_samples)
tokenizer.fit_on_texts(x)
sequences_1 = tokenizer.texts_to_sequences(x)

word_index_1 = (tokenizer.word_index)
num_encoder_tokens = len(word_index_1)
print('Found %s unique tokens.' % len(word_index_1))

tokenizer = Tokenizer(num_words=num_samples)
tokenizer.fit_on_texts(y)
sequences_2 = tokenizer.texts_to_sequences(y)

word_index_2 = tokenizer.word_index
num_decoder_tokens = len(word_index_2)
print('Found %s unique tokens.' % len(word_index_2))

data_1 = pad_sequences(sequences_1, maxlen=max_encoder_seq_length , padding='post')
data_2 = pad_sequences(sequences_2, maxlen=max_decoder_seq_length , padding= 'post')

print(data_1.shape)
print(data_2.shape)
print(num_encoder_tokens)
print(num_decoder_tokens)



250000


Using TensorFlow backend.


250000
['Presented by IM Pictures\n', 'In association with MVP Venture Capital and Cinema Service\n', 'My Sassy Girl\n', "We promised to meet here two years later, but she hasn't come yet.\n", 'Here we go.\n', 'One, two...\n', 'Hello?\n', "Sorry, I'm on my way.\n", "Yes, I'm coming.\n", 'Bye.\n']
['Produced by Shin Cine\n', 'Jeon Ji-hyun Cha Tae-hyun\n', 'Exactly two years ago today, she and I buried a time capsule here.\n', "I'm going to wait.\n", "Please, don't move.\n", 'Wait a minute.\n', 'Oh, auntie.\n', "I'm really sorry.\n", "I'm having my photo taken.\n", 'Are you ready?\n']
Found 24584 unique tokens.
Found 24569 unique tokens.
(250000, 15)
(250000, 15)
24584
24569


In [7]:
from keras.models import Model
from keras.layers import Input , Dense , GRU , LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
import numpy as np
import os
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
import tensorflow as tf
import random
from tqdm import tqdm
from babel.dates import format_date

import matplotlib.pyplot as plt
%matplotlib inline

embedding_dim = 50
Tx = max_encoder_seq_length
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/glove.6B/', 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index_1) + 1, embedding_dim))
for word, i in word_index_1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)
x = Embedding(len(word_index_1)+1 ,embedding_dim,weights=[embedding_matrix],input_length=max_encoder_seq_length,trainable=False)

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator(inputs =([a,s_prev]))
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context


post_activation_LSTM_cell = LSTM(128, return_state = True) # post-attention LSTM 
output_layer = Dense(24570, activation='softmax')

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):

    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    e = x(X)
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs = context,initial_state = [s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X,s0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

model = model(15 , 15 , 128, 128, 24584, 24569)

opt = Adam(lr = 0.005, beta_1 = 0.9,beta_2 = 0.999 , decay = 0.01)
model.compile(optimizer = opt , loss = "sparse_categorical_crossentropy" ,metrics = ['accuracy'])

s0 = np.zeros((250000, 128))
c0 = np.zeros((250000, 128))

data_2 = list(data_2.swapaxes(0,1))
model.fit([data_1, s0, c0], data_2, epochs=1, batch_size=10000)

Found 400000 word vectors.
(24585, 50)
Epoch 1/1
250000/250000 [==============================] - 2999s 12ms/step - loss: 78.2885 - dense_3_loss: 3.7666 - dense_3_accuracy: 0.0010 - dense_3_accuracy_1: 0.0993 - dense_3_accuracy_2: 0.2207 - dense_3_accuracy_3: 0.3448 - dense_3_accuracy_4: 0.4637 - dense_3_accuracy_5: 0.5702 - dense_3_accuracy_6: 0.6570 - dense_3_accuracy_7: 0.7246 - dense_3_accuracy_8: 0.7780 - dense_3_accuracy_9: 0.8156 - dense_3_accuracy_10: 0.8469 - dense_3_accuracy_11: 0.8714 - dense_3_accuracy_12: 0.8904 - dense_3_accuracy_13: 0.9050 - dense_3_accuracy_14: 0.9170
